<a href="https://colab.research.google.com/github/victwise/fastai_numerical_linear_algebra/blob/master/Cap%C3%ADtulo5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sources:

- https://eli.thegreenplace.net/2015/memory-layout-of-multi-dimensional-arrays/

In [0]:
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import math, scipy, numpy as np
from scipy import linalg

In [0]:
data = datasets.load_diabetes()

In [0]:
feature_names=['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

In [0]:
trn,test,y_trn,y_test = train_test_split(data.data, data.target, test_size=0.2)

In [5]:
trn.shape, test.shape

((353, 10), (89, 10))

# Regresión Lineal en Scikit Learn

In [6]:
regr = linear_model.LinearRegression()
%timeit regr.fit(trn, y_trn)

The slowest run took 95.08 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 510 µs per loop


In [0]:
pred = regr.predict(test)

Será útil tener algunas métricas sobre qué tan bueno es nuestro pronóstico. Veremos la norma de la media al cuadrado (L2) y el error absoluto de la media (L1).

In [0]:
def regr_metrics(act, pred):
    return (math.sqrt(metrics.mean_squared_error(act, pred)), 
     metrics.mean_absolute_error(act, pred))

In [10]:
regr_metrics(y_test, regr.predict(test))

(59.35040075820322, 48.05573038936139)

##Caracteristicas  Polinomiales

In [11]:
trn.shape

(353, 10)

In [0]:
poly = PolynomialFeatures(include_bias=False)


In [0]:
trn_feat = poly.fit_transform(trn)

In [14]:
', '.join(poly.get_feature_names(feature_names))

'age, sex, bmi, bp, s1, s2, s3, s4, s5, s6, age^2, age sex, age bmi, age bp, age s1, age s2, age s3, age s4, age s5, age s6, sex^2, sex bmi, sex bp, sex s1, sex s2, sex s3, sex s4, sex s5, sex s6, bmi^2, bmi bp, bmi s1, bmi s2, bmi s3, bmi s4, bmi s5, bmi s6, bp^2, bp s1, bp s2, bp s3, bp s4, bp s5, bp s6, s1^2, s1 s2, s1 s3, s1 s4, s1 s5, s1 s6, s2^2, s2 s3, s2 s4, s2 s5, s2 s6, s3^2, s3 s4, s3 s5, s3 s6, s4^2, s4 s5, s4 s6, s5^2, s5 s6, s6^2'

In [15]:
trn_feat.shape

(353, 65)

In [16]:
regr.fit(trn_feat, y_trn)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [17]:
regr_metrics(y_test, regr.predict(poly.fit_transform(test)))

(64.89628354688391, 50.96249290563969)

In [18]:
%timeit poly.fit_transform(trn)

1000 loops, best of 3: 618 µs per loop


##Experimentando con vecctorización y código nativo 

In [0]:
%matplotlib inline

In [25]:
!pip install pandas_summary

In [26]:
!pip install numba

    100% |████████████████████████████████| 3.2MB 7.5MB/s 
    100% |████████████████████████████████| 16.1MB 2.0MB/s 


In [0]:
import math, numpy as np, matplotlib.pyplot as plt
from pandas_summary import DataFrameSummary
from scipy import ndimage

In [0]:
from numba import jit, vectorize, guvectorize, cuda, float32, void, float64

Mostraremos el impacto de:

- Evitar asignaciones de memoria y copias (más lento que los cálculos de la CPU)
- Mejor localidad
- Vectorización

Si usamos numpy en arreglos completos a la vez, crea muchos temporarios y no podemos usar el caché. Si utilizamos un bucle numba a través de un elemento de matriz a la vez, no tenemos que asignar grandes matrices temporales, y podemos reutilizar los datos almacenados en caché, ya que estamos haciendo varios cálculos en cada elemento de la matriz.

In [0]:
# Untype and Unvectorized
def proc_python(xx,yy):
    zz = np.zeros(nobs, dtype='float32')
    for j in range(nobs):   
        x, y = xx[j], yy[j] 
        x = x*2 - ( y * 55 )
        y = x + y*2         
        z = x + y + 99      
        z = z * ( z - .88 ) 
        zz[j] = z           
    return zz

In [0]:
nobs = 10000
x = np.random.randn(nobs).astype('float32')
y = np.random.randn(nobs).astype('float32')

In [31]:
%timeit proc_python(x,y)   # Untyped and unvectorized

10 loops, best of 3: 76.4 ms per loop


##Numpy

In [0]:
# Typed and Vectorized
def proc_numpy(x,y):
    z = np.zeros(nobs, dtype='float32')
    x = x*2 - ( y * 55 )
    y = x + y*2         
    z = x + y + 99      
    z = z * ( z - .88 ) 
    return z

In [33]:
np.allclose( proc_numpy(x,y), proc_python(x,y), atol=1e-4 )

True

In [34]:
%timeit proc_numpy(x,y)    # Typed and vectorized

The slowest run took 50.67 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 30.4 µs per loop


##Numba

Numba ofrece varios decoradores diferentes. Intentaremos dos diferentes:

- @jit: muy general
- @vectorize: no es necesario escribir un bucle for.

Útil cuando se opera en vectores del mismo tamaño.
Primero, usaremos el decorador del compilador jit (just-in-time) de Numba, sin vectorizar explícitamente. Esto evita grandes asignaciones de memoria, por lo que tenemos una mejor localidad:

In [0]:
@jit()
def proc_numba(xx,yy,zz):
    for j in range(nobs):   
        x, y = xx[j], yy[j] 
        x = x*2 - ( y * 55 )
        y = x + y*2         
        z = x + y + 99      
        z = z * ( z - .88 ) 
        zz[j] = z           
    return zz

In [36]:
z = np.zeros(nobs).astype('float32')
np.allclose( proc_numpy(x,y), proc_numba(x,y,z), atol=1e-4 )

True

In [37]:
%timeit proc_numba(x,y,z)

The slowest run took 5.14 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 8.75 µs per loop


Ahora usaremos el decorador **vectorize** de Numba. El compilador de Numba optimiza esto de una manera más inteligente que lo que es posible con Python y Numpy.

In [0]:
@vectorize
def vec_numba(x,y):
    x = x*2 - ( y * 55 )
    y = x + y*2         
    z = x + y + 99      
    return z * ( z - .88 ) 
  

In [39]:
np.allclose(vec_numba(x,y), proc_numba(x,y,z), atol=1e-4 )

True

In [40]:
%timeit vec_numba(x,y)

The slowest run took 47.93 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 7.91 µs per loop


##Numba - caracteristicas polinomiales

De la entrada del blog de Eli Bendersky :

"El diseño de la fila principal de una matriz, coloca la primera fila en la memoria contigua, luego la segunda fila justo después de ella, luego la tercera, y así sucesivamente. El diseño de la columna principal coloca la primera columna en la memoria contigua, luego la segunda, etc. .... Si bien saber qué diseño está utilizando un conjunto de datos en particular es fundamental para un buen rendimiento, no hay una respuesta única a la pregunta de qué diseño 'es mejor' en general.

"Resulta que la coincidencia con la forma en que funciona su algoritmo con el diseño de datos puede hacer o deshacer el rendimiento de una aplicación.

"El corto para llevar es: recorrer siempre los datos en el orden en que se diseñaron ".

Diseño de columna principal : Fortran, Matlab, R y Julia

Diseño de la fila principal : C, C ++, Python, Pascal, Mathematica

In [0]:
@jit(nopython=True)
def vec_poly(x, res):
    m,n=x.shape
    feat_idx=0
    for i in range(n):
        v1=x[:,i]
        for k in range(m): res[k,feat_idx] = v1[k]
        feat_idx+=1
        for j in range(i,n):
            for k in range(m): res[k,feat_idx] = v1[k]*x[k,j]
            feat_idx+=1


Row-Major vs Column-Major Storage

In [0]:
trn = np.asfortranarray(trn)
test = np.asfortranarray(test)

In [0]:
m,n=trn.shape
n_feat = n*(n+1)//2 + n
trn_feat = np.zeros((m,n_feat), order='F')
test_feat = np.zeros((len(y_test), n_feat), order='F')

In [0]:
vec_poly(trn, trn_feat)
vec_poly(test, test_feat)

In [45]:
regr.fit(trn_feat, y_trn)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [46]:
regr_metrics(y_test, regr.predict(test_feat))

(64.89628354688381, 50.96249290563954)

In [47]:
%timeit vec_poly(trn, trn_feat)

The slowest run took 7.05 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 8.16 µs per loop


In [48]:
%timeit poly.fit_transform(trn)

The slowest run took 6.77 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 596 µs per loop


In [49]:
605/7.7

78.57142857142857

¡Este es un gran avance! Numba es increíble ! Con una sola línea de código, estamos obteniendo una aceleración de 78x sobre scikit learn (que fue optimizada por expertos).

##Regularización y ruido 

La regularización es una forma de reducir el ajuste excesivo y crear modelos que se generalicen mejor a nuevos datos.

Regularización 

La regresión de lazo utiliza una penalización de L1, que empuja hacia coeficientes dispersos. El parámetro se utiliza para ponderar el término de la penalidad. Lasso CV de Scikit Learn realiza una validación cruzada con varios valores diferentes para α.


In [0]:
reg_regr = linear_model.LassoCV(n_alphas=10)

In [51]:
reg_regr.fit(trn_feat, y_trn)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=10, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [52]:
reg_regr.alpha_

0.010299084881356342

In [53]:
regr_metrics(y_test, reg_regr.predict(test_feat))

(57.763607562678985, 46.73144756229311)

##Ruido

Ahora añadimos algo de ruido a la Data.

In [0]:
idxs = np.random.randint(0, len(trn), 10)

In [0]:
y_trn2 = np.copy(y_trn)
y_trn2[idxs] *= 10 # label noise

In [56]:
regr = linear_model.LinearRegression()
regr.fit(trn, y_trn)
regr_metrics(y_test, regr.predict(test))

(59.350400758203214, 48.055730389361386)

In [57]:
regr.fit(trn, y_trn2)
regr_metrics(y_test, regr.predict(test))

(78.56285761014834, 61.66400253944768)

La pérdida de Huber es una función de pérdida que es menos sensible a los valores atípicos que la pérdida por error al cuadrado. Es cuadrático para valores de error pequeños y lineal para valores grandes.

In [58]:
hregr = linear_model.HuberRegressor()
hregr.fit(trn, y_trn2)
regr_metrics(y_test, hregr.predict(test))

(59.45305509008837, 48.01953043044526)